## p.145- 계층적 인덱싱 (다중 인덱싱)

In [ ]:
# 단일 인덱스 내에 여러 인덱스 레벨을 포함

In [1]:
import pandas as pd
import numpy as np

In [2]:
index = [('Cal',2000),('Cal',2010),
         ('NY',2000),('NY',2010),
         ('Tex',2000),('Tex',2010)]
population = [33871648, 37253956,
              18976457, 19378102,
              20851820, 25145561]
pop = pd.Series(population, index=index)
pop

(Cal, 2000)    33871648
(Cal, 2010)    37253956
(NY, 2000)     18976457
(NY, 2010)     19378102
(Tex, 2000)    20851820
(Tex, 2010)    25145561
dtype: int64

In [3]:
pop[('Cal',2010):('Tex',2000)] # --> (Cal, 2010) 자체가 인덱스가 되어버려서 효율적이지 않음

(Cal, 2010)    37253956
(NY, 2000)     18976457
(NY, 2010)     19378102
(Tex, 2000)    20851820
dtype: int64

In [10]:
pop[[i for i in pop.index if i[1] == 2010]] # --> 효율적이지 않음

Cal  2010    37253956
NY   2010    19378102
Tex  2010    25145561
dtype: int64

#### 더 나은 방식 (PANDAS MULTIINDEX)

In [17]:
index = pd.MultiIndex.from_tuples(index) # 튜플의 리스트로부터 멀티인덱스를 만듦
index

MultiIndex([('Cal', 2000),
            ('Cal', 2010),
            ( 'NY', 2000),
            ( 'NY', 2010),
            ('Tex', 2000),
            ('Tex', 2010)],
           )

In [7]:
# MultiIndex를 시리즈로 다시 인덱싱 -> 데이터의 계층적 표현 가능
pop = pop.reindex(index)
pop

Cal  2000    33871648
     2010    37253956
NY   2000    18976457
     2010    19378102
Tex  2000    20851820
     2010    25145561
dtype: int64

In [8]:
pop[:,2010] # --> 효율적

Cal    37253956
NY     19378102
Tex    25145561
dtype: int64

#### - unstack() : 다중인덱스를 가진 시리즈를 전형적인덱스를 가진 DATAFRAME으로 변경
#### - stack() : 반대로

In [11]:
pop_df = pop.unstack() # 두번째 인덱스를 열로 만들어줌
pop_df

,2000,2010
Cal,33871648,37253956
NY,18976457,19378102
Tex,20851820,25145561


In [12]:
pop_df.stack()

Cal  2000    33871648
     2010    37253956
NY   2000    18976457
     2010    19378102
Tex  2000    20851820
     2010    25145561
dtype: int64

In [14]:
# 데이터(under18) 추가
pop_df = pd.DataFrame({'total':pop,
                    'under18':[9267089, 9284094,
                               4687374, 4318033,
                               5906301, 68769014]})
pop_df 

total   under18
Cal 2000  33871648   9267089
    2010  37253956   9284094
NY  2000  18976457   4687374
    2010  19378102   4318033
Tex 2000  20851820   5906301
    2010  25145561  68769014

In [16]:
f_u18 = pop_df['under18']/pop_df['total']
f_u18.unstack()

,2000,2010
Cal,0.273594,0.249211
NY,0.247010,0.222831
Tex,0.283251,2.734837


#### MULTIINDEX 생성메서드

In [27]:
df = pd.DataFrame(np.random.rand(4,2),
                 index = [['a','a','b','b'],[1,2,1,2]], # MULTIINDEX
                 columns = ['data1','data2'])
df

data1     data2
a 1  0.125377  0.354247
  2  0.421694  0.284273
b 1  0.834300  0.428753
  2  0.110548  0.202627

In [28]:
data = {('Cal',2000) : 33871648,
        ('Cal',2010) : 37253956,
        ('NY',2000) : 18976457,
        ('NY',2010) : 19378102,
        ('Tex',2000) : 20851820,
        ('Tex',2010) : 25145561}
pd.Series(data)
# 튜플을 키로 갖는 딕셔너리를 전달하면 pandas가 자동으로 인식해서 MULTIINDEX를 사용함

Cal  2000    33871648
     2010    37253956
NY   2000    18976457
     2010    19378102
Tex  2000    20851820
     2010    25145561
dtype: int64

#### 명시적 MULTIINDEX 생성자

In [33]:
pd.MultiIndex.from_product([['a','b'],[1,2]])

MultiIndex([('a', 1),
            ('a', 2),
            ('b', 1),
            ('b', 2)],
           )

In [31]:
pop.index.names = ['state','year'] # 라벨 이름 지정
pop

state  year
Cal    2000    33871648
       2010    37253956
NY     2000    18976457
       2010    19378102
Tex    2000    20851820
       2010    25145561
dtype: int64

#### MULTIINDEX 인덱싱 및 슬라이싱

In [35]:
pop

state  year
Cal    2000    33871648
       2010    37253956
NY     2000    18976457
       2010    19378102
Tex    2000    20851820
       2010    25145561
dtype: int64

In [36]:
pop['Cal',2000]

33871648

In [37]:
pop['Cal']

year
2000    33871648
2010    37253956
dtype: int64

In [38]:
pop.loc['Cal':'NY']

state  year
Cal    2000    33871648
       2010    37253956
NY     2000    18976457
       2010    19378102
dtype: int64

In [40]:
pop>22000000 # 이건 T/F값이니까

state  year
Cal    2000     True
       2010     True
NY     2000    False
       2010    False
Tex    2000    False
       2010     True
dtype: bool

In [39]:
pop[pop>22000000] # 여기서는 T/F값 중 T인 값의 pop데이터를 가져오도록 함

state  year
Cal    2000    33871648
       2010    37253956
Tex    2010    25145561
dtype: int64

In [41]:
pop[['Cal','Tex']] # 펜시인덱싱도 가능

state  year
Cal    2000    33871648
       2010    37253956
Tex    2000    20851820
       2010    25145561
dtype: int64

In [51]:
pop_df 

total   under18
state year                    
Cal   2000  33871648   9267089
      2010  37253956   9284094
NY    2000  18976457   4687374
      2010  19378102   4318033
Tex   2000  20851820   5906301
      2010  25145561  68769014

In [52]:
pop_df.iloc[1:3,0]

state  year
Cal    2010    37253956
NY     2000    18976457
Name: total, dtype: int64

#### 다중인덱스를 가진 DATAFRAME

In [43]:
#  행 인덱스 생성 (index):
index = pd.MultiIndex.from_product([[2013,2014],[1,2]], # 외부 레벨 (year)은 2013과 2014로 구성, 내부 레벨 (visit)은 1과 2로 구성
                                    names = ['year','visit'])
# 열 인덱스 생성 (columns):
columns = pd.MultiIndex.from_product([['Bob','Guido','Sue'],['HR','Temp']],
                                     names = ['subject','type'])
data = np.round(np.random.randn(4,6),1) # np.random.randn : 평균이 0이고 표준편차가 1인 표준 정규분포
data[:,::2] *= 10 # 짝수 열에는 값을 10배로 증가시킴
data += 37

# 데이터 생성 및 데이터 프레임 생성 (health_data):
health_data = pd.DataFrame(data,index=index, columns=columns)
health_data

subject      Bob       Guido         Sue      
type          HR  Temp    HR  Temp    HR  Temp
year visit                                    
2013 1      37.0  35.5  10.0  36.2  24.0  36.2
     2      31.0  38.3  42.0  37.3  30.0  35.7
2014 1      53.0  37.9  61.0  37.8  59.0  37.8
     2      24.0  38.6  20.0  36.0  22.0  34.8

In [44]:
health_data['Guido','HR']

year  visit
2013  1        10.0
      2        42.0
2014  1        61.0
      2        20.0
Name: (Guido, HR), dtype: float64

In [46]:
health_data.loc[:,('Bob','HR')] 
# loc, iloc 사용가능하지만, 이런 인덱스 튜플 내에서 슬라이싱하려고 하면 err

year  visit
2013  1        37.0
      2        31.0
2014  1        53.0
      2        24.0
Name: (Bob, HR), dtype: float64

In [48]:
# 대신 IndexSilce 객체사용 (idx로 간단하게 정의해서 사용했음)
idx = pd.IndexSlice
health_data.loc[idx[:,1],idx[:,'HR']]

,subject,Bob,Guido,Sue
,type,HR,HR,HR
year,visit,,,
2013,1,37.0,10.0,24.0
2014,1,53.0,61.0,59.0


#### 다중인덱스 재정렬하기 (sort_index())

In [65]:
index = pd.MultiIndex.from_product([['a','c','b'],[1,2]])
data = pd.Series(np.random.rand(6), index=index)
data.index.names = ['char','int']
data

char  int
a     1      0.925515
      2      0.800040
c     1      0.948880
      2      0.825754
b     1      0.483122
      2      0.813630
dtype: float64

In [66]:
try:
    data['a':'b']
except KeyErrror as e:
    print(type(e))
    print(e)
# --> 정렬되어있지 않아서 err

NameError: name 'KeyErrror' is not defined

In [67]:
data = data.sort_index()
data

char  int
a     1      0.925515
      2      0.800040
b     1      0.483122
      2      0.813630
c     1      0.948880
      2      0.825754
dtype: float64

In [68]:
data['a':'b']

char  int
a     1      0.925515
      2      0.800040
b     1      0.483122
      2      0.813630
dtype: float64

#### 인덱스 스태킹, 언스태킹

In [69]:
pop.unstack(level=0)

state,Cal,NY,Tex
year,,,
2000,33871648,18976457,20851820
2010,37253956,19378102,25145561


In [70]:
pop.unstack(level=1)

year,2000,2010
state,,
Cal,33871648,37253956
NY,18976457,19378102
Tex,20851820,25145561


#### ! 인덱스 설정 및 재설정 (reset_index) ! -- 많이 사용

In [72]:
# reset_index : 인덱스 레이블을 열로 바꿈
pop_flat = pop.reset_index(name="population")
pop_flat

,state,year,population
0,Cal,2000,33871648
1,Cal,2010,37253956
2,NY,2000,18976457
3,NY,2010,19378102
4,Tex,2000,20851820
5,Tex,2010,25145561


In [73]:
pop_flat.set_index(['state','year'])

population
state year            
Cal   2000    33871648
      2010    37253956
NY    2000    18976457
      2010    19378102
Tex   2000    20851820
      2010    25145561

## p.161- 데이터세트 결합 - pd.concat()

In [75]:
def make_df(cols,ind):
    """빠르게 DataFrame 생성"""
    data = {c:[str(c) + str(i) for i in ind]
        for c in cols}
    return pd.DataFrame(data, ind)

In [76]:
make_df('ABC',range(3))

,A,B,C
0,A0,B0,C0
1,A1,B1,C1
2,A2,B2,C2


In [95]:
class display(object):
    """Display HTML representation of multiple objects"""
    template = """<div style="float: left; padding: 10px;">
    <p style = 'font-family:"Courier New", Courier, monospace'>{0}{1}
    """
    def __init__(self, *args):
        self.args = args
    def __repr_html_(self):
        return '\n'.join(self.template.format(a,eval(a)._repr_html_())
        for a in self.args)
    def __repr__(self):
        return '\n\n'.join(a+'\n'+repr(eval(a))
        for a in self.args)

In [79]:
x=[1,2,3]
y=[4,5,6]
z=[7,8,9]
np.concatenate([x,y,z])

array([1, 2, 3, 4, 5, 6, 7, 8, 9])

In [80]:
x=[[1,2],
   [3,4]]
np.concatenate([x,x],axis=1)

array([[1, 2, 1, 2],
       [3, 4, 3, 4]])

#### np.concatenate는 배열을 간단하게 연결
#### pd.concat()은 series나 dataframe을 간단하게 연결 (판다스는 index를 잡음)

In [81]:
ser1 = pd.Series(['A','B','C'], index=[1,2,3])
ser2 = pd.Series(['D','E','F'], index=[1,2,3])

In [83]:
data = pd.concat([ser1,ser2])
data

1    A
2    B
3    C
1    D
2    E
3    F
dtype: object

In [84]:
data.reset_index() # -- .reset_index()으로 인덱스 재정의가능

,index,0
0,1,A
1,2,B
2,3,C
3,1,D
4,2,E
5,3,F


In [85]:
pd.concat([ser1,ser2],ignore_index=True) # -- ignore_index=True 쓸수도 있음

0    A
1    B
2    C
3    D
4    E
5    F
dtype: object

In [87]:
df1 = make_df('AB',[1,2])
df2 = make_df('AB',[3,4])
data = pd.concat([df1,df2])
data

,A,B
1,A1,B1
2,A2,B2
3,A3,B3
4,A4,B4


In [91]:
df5 = make_df('ABC',[1,2])
df6 = make_df('BCD',[3,4])
print(df5); print(df6); print(pd.concat([df5,df6]))

    A   B   C
1  A1  B1  C1
2  A2  B2  C2
    B   C   D
3  B3  C3  D3
4  B4  C4  D4
     A   B   C    D
1   A1  B1  C1  NaN
2   A2  B2  C2  NaN
3  NaN  B3  C3   D3
4  NaN  B4  C4   D4


## 데이터세트 결합 (병합, 조인) - pd.merge()

In [96]:
# 일대일조인
df1 = pd.DataFrame({'employee': ['Bob','Jale','Lisa','Sue'],
                    'group' : ['Accounting','Engineering','Engineering','HR']})
df2 = pd.DataFrame({'employee': ['Bob','Jale','Lisa','Sue'],
                    'hire_date' : [2004,2008,2012,2014]})
print(df1); print(df2)

  employee        group
0      Bob   Accounting
1     Jale  Engineering
2     Lisa  Engineering
3      Sue           HR
  employee  hire_date
0      Bob       2004
1     Jale       2008
2     Lisa       2012
3      Sue       2014


In [98]:
df3 = pd.merge(df1,df2)
df3

,employee,group,hire_date
0,Bob,Accounting,2004
1,Jale,Engineering,2008
2,Lisa,Engineering,2012
3,Sue,HR,2014


In [102]:
# 다대일조인
df4 = pd.DataFrame({'group' : ['Accounting','Engineering','HR'],
                    'supervisor' : ['Carly','Guido','Steve']})
print(df3); print(df4); print(pd.merge(df3,df4))

  employee        group  hire_date
0      Bob   Accounting       2004
1     Jale  Engineering       2008
2     Lisa  Engineering       2012
3      Sue           HR       2014
         group supervisor
0   Accounting      Carly
1  Engineering      Guido
2           HR      Steve
  employee        group  hire_date supervisor
0      Bob   Accounting       2004      Carly
1     Jale  Engineering       2008      Guido
2     Lisa  Engineering       2012      Guido
3      Sue           HR       2014      Steve


In [103]:
# 다대다조인
df5 = pd.DataFrame({'group' : ['Accounting','Accounting','Engineering','Engineering','HR','HR'],
                    'skills' : ['math','spreadsheets','coding','linux','spreadsheets','organization']})
print(df1); print(df5); print(pd.merge(df1,df5))

  employee        group
0      Bob   Accounting
1     Jale  Engineering
2     Lisa  Engineering
3      Sue           HR
         group        skills
0   Accounting          math
1   Accounting  spreadsheets
2  Engineering        coding
3  Engineering         linux
4           HR  spreadsheets
5           HR  organization
  employee        group        skills
0      Bob   Accounting          math
1      Bob   Accounting  spreadsheets
2     Jale  Engineering        coding
3     Jale  Engineering         linux
4     Lisa  Engineering        coding
5     Lisa  Engineering         linux
6      Sue           HR  spreadsheets
7      Sue           HR  organization


## pd.merge()가 제공하는 옵션

In [106]:
print(df1); print(df2); print(pd.merge(df1,df2, on="employee"))

  employee        group
0      Bob   Accounting
1     Jale  Engineering
2     Lisa  Engineering
3      Sue           HR
  employee  hire_date
0      Bob       2004
1     Jale       2008
2     Lisa       2012
3      Sue       2014
  employee        group  hire_date
0      Bob   Accounting       2004
1     Jale  Engineering       2008
2     Lisa  Engineering       2012
3      Sue           HR       2014


In [107]:
df3 = pd.DataFrame({'name':['Bob','Jake','Lisa','Sue'],
                    'salary' : [70000,80000,120000,90000]})
print(df1); print(df3); print(pd.merge(df1,df3, left_on="employee", right_on="name"))
# 이러면 left on, right on 두개가 중복된 열을 가지게 됨

  employee        group
0      Bob   Accounting
1     Jale  Engineering
2     Lisa  Engineering
3      Sue           HR
   name  salary
0   Bob   70000
1  Jake   80000
2  Lisa  120000
3   Sue   90000
  employee        group  name  salary
0      Bob   Accounting   Bob   70000
1     Lisa  Engineering  Lisa  120000
2      Sue           HR   Sue   90000


In [108]:
pd.merge(df1,df3, left_on="employee", right_on="name").drop('name',axis=1)

,employee,group,salary
0,Bob,Accounting,70000
1,Lisa,Engineering,120000
2,Sue,HR,90000


In [121]:
pops = pd.read_csv('python-ds-handbook/notebooks/data/state-population.csv', header=0, names=['abbreviation','ages','year','population'])
# 이렇게 이름을 미리 맞춰주면 merge할때 편함
areas = pd.read_csv('python-ds-handbook/notebooks/data/state-areas.csv')
states = pd.read_csv('python-ds-handbook/notebooks/data/state-abbrevs.csv')

print(pop.head()); print(aread.head()); print(abbrevs.head()); 

  state/region     ages  year  population
0           AL  under18  2012   1117489.0
1           AL    total  2012   4817528.0
2           AL  under18  2010   1130966.0
3           AL    total  2010   4785570.0
4           AL  under18  2011   1125763.0
        state  area (sq. mi)
0     Alabama          52423
1      Alaska         656425
2     Arizona         114006
3    Arkansas          53182
4  California         163707
        state abbreviation
0     Alabama           AL
1      Alaska           AK
2     Arizona           AZ
3    Arkansas           AR
4  California           CA


In [122]:
pd.merge(states,areas)

,state,abbreviation,area (sq. mi)
0,Alabama,AL,52423
1,Alaska,AK,656425
2,Arizona,AZ,114006
3,Arkansas,AR,53182
4,California,CA,163707
5,Colorado,CO,104100
6,Connecticut,CT,5544
7,Delaware,DE,1954
8,District of Columbia,DC,68
9,Florida,FL,65758


In [123]:
pd.merge(df,pops)

,state,abbreviation,area (sq. mi),ages,year,population
0,Alabama,AL,52423,under18,2012,1117489.0
1,Alabama,AL,52423,total,2012,4817528.0
2,Alabama,AL,52423,under18,2010,1130966.0
3,Alabama,AL,52423,total,2010,4785570.0
4,Alabama,AL,52423,under18,2011,1125763.0
...,...,...,...,...,...,...
2443,Wyoming,WY,97818,under18,1993,137458.0
2444,Wyoming,WY,97818,total,1991,459260.0
2445,Wyoming,WY,97818,under18,1991,136720.0
2446,Wyoming,WY,97818,under18,1990,136078.0


In [124]:
states.shape, areas.shape, pd.merge(states,areas).shape # how : null값을 어떻게 처리할지

((51, 2), (52, 2), (51, 3))

In [130]:
states.shape, areas.shape, pd.merge(states, areas, how="inner").shape

((51, 2), (52, 2), (51, 3))

In [126]:
states.shape, areas.shape, pd.merge(states, areas, how="outer").shape

((51, 2), (52, 2), (52, 3))

In [128]:
states.shape, areas.shape, pd.merge(states, areas, how="left").shape

((51, 2), (52, 2), (51, 3))

In [129]:
states.shape, areas.shape, pd.merge(states, areas, how="right").shape

((51, 2), (52, 2), (52, 3))

In [131]:
states.shape, areas.shape, pd.merge(states, areas, how="outer").isnull().any()

((51, 2),
 (52, 2),
 state            False
 abbreviation      True
 area (sq. mi)    False
 dtype: bool)

In [135]:
merge_df = pd.merge(states,areas)

In [136]:
merge_df['abbreviation'].unique(), pops['abbreviation'].unique()

(array(['AL', 'AK', 'AZ', 'AR', 'CA', 'CO', 'CT', 'DE', 'DC', 'FL', 'GA',
        'HI', 'ID', 'IL', 'IN', 'IA', 'KS', 'KY', 'LA', 'ME', 'MT', 'NE',
        'NV', 'NH', 'NJ', 'NM', 'NY', 'NC', 'ND', 'OH', 'OK', 'OR', 'MD',
        'MA', 'MI', 'MN', 'MS', 'MO', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX',
        'UT', 'VT', 'VA', 'WA', 'WV', 'WI', 'WY'], dtype=object),
 array(['AL', 'AK', 'AZ', 'AR', 'CA', 'CO', 'CT', 'DE', 'DC', 'FL', 'GA',
        'HI', 'ID', 'IL', 'IN', 'IA', 'KS', 'KY', 'LA', 'ME', 'MD', 'MA',
        'MI', 'MN', 'MS', 'MO', 'MT', 'NE', 'NV', 'NH', 'NJ', 'NM', 'NY',
        'NC', 'ND', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX',
        'UT', 'VT', 'VA', 'WA', 'WV', 'WI', 'WY', 'PR', 'USA'],
       dtype=object))

## p.187- GroupBy (분할,적용,결합)

In [139]:
df = pd.DataFrame({'key':['A','B','C','A','B','C'],
                   'data' : range(6)}, columns = ['key','data'])
df

,key,data
0,A,0
1,B,1
2,C,2
3,A,3
4,B,4
5,C,5


In [140]:
df.groupby('key')

In [141]:
df.groupby('key').sum()

,data
key,
A,3
B,5
C,7


In [144]:
import seaborn as sns
planets = sns.load_dataset('planets')
planets.shape

(1035, 6)

In [170]:
planets.head(); planets.info(); planets.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1035 entries, 0 to 1034
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   method          1035 non-null   object 
 1   number          1035 non-null   int64  
 2   orbital_period  992 non-null    float64
 3   mass            513 non-null    float64
 4   distance        808 non-null    float64
 5   year            1035 non-null   int64  
dtypes: float64(3), int64(2), object(1)
memory usage: 48.6+ KB


,number,orbital_period,mass,distance,year
count,1035.000000,992.000000,513.000000,808.000000,1035.000000
mean,1.785507,2002.917596,2.638161,264.069282,2009.070531
std,1.240976,26014.728304,3.818617,733.116493,3.972567
min,1.000000,0.090706,0.003600,1.350000,1989.000000
25%,1.000000,5.442540,0.229000,32.560000,2007.000000
50%,1.000000,39.979500,1.260000,55.250000,2010.000000
75%,2.000000,526.005000,3.040000,178.500000,2012.000000
max,7.000000,730000.000000,25.000000,8500.000000,2014.000000


In [171]:
planets.groupby('method')

<bound method GroupBy.mean of <pandas.core.groupby.generic.DataFrameGroupBy object at 0x000001F9A7536D00>>

In [173]:
planets.groupby('method')['orbital_period'].mean()

method
Astrometry                          631.180000
Eclipse Timing Variations          4751.644444
Imaging                          118247.737500
Microlensing                       3153.571429
Orbital Brightness Modulation         0.709307
Pulsar Timing                      7343.021201
Pulsation Timing Variations        1170.000000
Radial Velocity                     823.354680
Transit                              21.102073
Transit Timing Variations            79.783500
Name: orbital_period, dtype: float64

In [148]:
planets.groupby('method')['orbital_period'].median()

method
Astrometry                         631.180000
Eclipse Timing Variations         4343.500000
Imaging                          27500.000000
Microlensing                      3300.000000
Orbital Brightness Modulation        0.342887
Pulsar Timing                       66.541900
Pulsation Timing Variations       1170.000000
Radial Velocity                    360.200000
Transit                              5.714932
Transit Timing Variations           57.011000
Name: orbital_period, dtype: float64

In [174]:
planets.groupby('method')['orbital_period'].describe()

,count,mean,std,min,25%,50%,75%,max
method,,,,,,,,
Astrometry,2.0,631.180000,544.217663,246.360000,438.770000,631.180000,823.590000,1016.000000
Eclipse Timing Variations,9.0,4751.644444,2499.130945,1916.250000,2900.000000,4343.500000,5767.000000,10220.000000
Imaging,12.0,118247.737500,213978.177277,4639.150000,8343.900000,27500.000000,94250.000000,730000.000000
Microlensing,7.0,3153.571429,1113.166333,1825.000000,2375.000000,3300.000000,3550.000000,5100.000000
Orbital Brightness Modulation,3.0,0.709307,0.725493,0.240104,0.291496,0.342887,0.943908,1.544929
Pulsar Timing,5.0,7343.021201,16313.265573,0.090706,25.262000,66.541900,98.211400,36525.000000
Pulsation Timing Variations,1.0,1170.000000,NaN,1170.000000,1170.000000,1170.000000,1170.000000,1170.000000
Radial Velocity,553.0,823.354680,1454.926210,0.736540,38.021000,360.200000,982.000000,17337.500000
Transit,397.0,21.102073,46.185893,0.355000,3.160630,5.714932,16.145700,331.600590


In [149]:
for (method,group) in planets.groupby('method'):
    print("{0:30s} shape={1}".format(method,group.shape))

Astrometry                     shape=(2, 6)
Eclipse Timing Variations      shape=(9, 6)
Imaging                        shape=(38, 6)
Microlensing                   shape=(23, 6)
Orbital Brightness Modulation  shape=(3, 6)
Pulsar Timing                  shape=(5, 6)
Pulsation Timing Variations    shape=(1, 6)
Radial Velocity                shape=(553, 6)
Transit                        shape=(397, 6)
Transit Timing Variations      shape=(4, 6)


In [151]:
planets.groupby('method')['year'].describe().unstack()

       method                       
count  Astrometry                          2.0
       Eclipse Timing Variations           9.0
       Imaging                            38.0
       Microlensing                       23.0
       Orbital Brightness Modulation       3.0
                                         ...  
max    Pulsar Timing                    2011.0
       Pulsation Timing Variations      2007.0
       Radial Velocity                  2014.0
       Transit                          2014.0
       Transit Timing Variations        2014.0
Length: 80, dtype: float64

In [ ]:
# aggregate() : 문자열, 함수, 리스트 등에 모든 집계를 계산가능

In [166]:
rng = np.random.RandomState(0)
df = pd.DataFrame({'key' : ['A','B','C','A','B','C'],
                   'data1' : range(6),
                   'data2' : rng.randint(0,10,6)},
                  columns = ['key','data1','data2'])
df

,key,data1,data2
0,A,0,5
1,B,1,0
2,C,2,3
3,A,3,3
4,B,4,7
5,C,5,9


In [167]:
df.groupby('key').aggregate(['min','median','max'])

data1            data2           
      min median max   min median max
key                                  
A       0    1.5   3     3    4.0   5
B       1    2.5   4     0    3.5   7
C       2    3.5   5     3    6.0   9

In [168]:
df.groupby('key').aggregate({'data1':'min',
                             'data2':'max'})

,data1,data2
key,,
A,0,5
B,1,7
C,2,9


In [176]:
df.groupby('method').aggregate([{'orbital_period':'mean','number':'sum'}) # 각각 다른 연산 줄 수 있음

KeyError: 'method'

In [169]:
def filter_func(x):
    return x['data2'].std() > 4